In [10]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [12]:
import torch

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 빌드 버전: {torch.version.cuda}") # PyTorch가 빌드된 CUDA 버전

if torch.cuda.is_available():
    print("\n-------------------------------------------")
    print("      🎉🎉 GPU를 사용할 수 있습니다! 🎉🎉")
    print("-------------------------------------------")
    print(f"  현재 GPU 장치: {torch.cuda.get_device_name(0)}")
    print(f"  GPU 장치 개수: {torch.cuda.device_count()}")
    device = torch.device("cuda") # GPU 사용
    
    # 간단한 GPU 연산 테스트
    print("\n  GPU에서 텐서 연산 테스트 중...")
    x = torch.rand(5, 5).to(device)
    y = torch.rand(5, 5).to(device)
    z = x + y
    print(f"  결과 텐서의 장치: {z.device}")
    print(f"  GPU 연산 성공!")
    
else:
    print("\n-------------------------------------------")
    print("      😭 GPU를 사용할 수 없습니다. 😭")
    print("-------------------------------------------")
    print("  CUDA 빌드 버전이 None이거나, torch.cuda.is_available()이 False입니다.")
    print("  NVIDIA 드라이버, CUDA Toolkit, cuDNN 설치 상태를 다시 확인해주세요.")
    device = torch.device("cpu") # CPU 사용

print(f"\n모델 및 데이터에 사용할 최종 장치: {device}")

PyTorch 버전: 2.7.1+cu118
CUDA 빌드 버전: 11.8

-------------------------------------------
      🎉🎉 GPU를 사용할 수 있습니다! 🎉🎉
-------------------------------------------
  현재 GPU 장치: NVIDIA RTX A4000
  GPU 장치 개수: 1

  GPU에서 텐서 연산 테스트 중...
  결과 텐서의 장치: cuda:0
  GPU 연산 성공!

모델 및 데이터에 사용할 최종 장치: cuda


In [14]:
import torch

# GPU 사용 가능하면 True, 아니면 False
print("CUDA available:", torch.cuda.is_available())

# GPU가 몇 개나 있는지도 확인
print("GPU device count:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("First GPU name:", torch.cuda.get_device_name(0))
    import torch
print(torch.cuda.is_available(), torch.cuda.device_count())


CUDA available: True
GPU device count: 1
First GPU name: NVIDIA RTX A4000
True 1


In [16]:
import torch
print(torch.__version__)
print(torch.version.cuda) # PyTorch가 빌드된 CUDA 버전

2.7.1+cu118
11.8


In [18]:
from transformers import AutoTokenizer, BertForSequenceClassification

# use_fast=False 로 slow tokenizer 사용
tokenizer = AutoTokenizer.from_pretrained(
    "skt/kobert-base-v1",
    use_fast=False
)
model = BertForSequenceClassification.from_pretrained(
    "skt/kobert-base-v1",
    num_labels=15
)

print("✅ slow tokenizer 로드 완료")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ slow tokenizer 로드 완료


In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# ① 분류기 로드
classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli",
    device=-1
)

labels = [
    '미백_긍정','미백_중립','미백_부정',
    '보습_긍정','보습_중립','보습_부정',
    '트러블_긍정','트러블_중립','트러블_부정',
    '피부_긍정','피부_중립','피부_부정',
    '노화_긍정','노화_중립','노화_부정'
]

# ② 리뷰 CSV 로드 (컬럼명은 'text' 여야 합니다)
df = pd.read_csv("raw_reviews.csv")

# ③ 분류 함수
def zsl_label(text, threshold=0.5):
    out = classifier(text, candidate_labels=labels, multi_label=True)
    # score > threshold 면 1, 아니면 0
    return { lab: int(score>threshold) 
             for lab, score in zip(out['labels'], out['scores']) }

# ④ 모든 리뷰에 적용
tqdm.pandas(desc="Zero-Shot 라벨링")
labels_df = df['text'].progress_apply(zsl_label).apply(pd.Series)

# ⑤ 원본 + 라벨 합치기
df_labeled = pd.concat([df, labels_df], axis=1)

# ⑥ 결과 저장
df_labeled.to_csv("raw_reviews_zero_shot_labeled.csv", index=False)
print("✅ 완료: raw_reviews_zero_shot_labeled.csv 생성됨")


In [ ]:
import os
import glob
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# 1) 작업 디렉토리로 이동 (파일이 있는 폴더)
os.chdir(r"C:\Users\User\Desktop\skindata")

# 2) Zero-Shot 분류기 로드
classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli",
    device=0  # GPU 사용 시 0으로 변경
)

# 3) 라벨 후보 목록 (15개)
candidate_labels = [
    '미백_긍정','미백_중립','미백_부정',
    '보습_긍정','보습_중립','보습_부정',
    '트러블_긍정','트러블_중립','트러블_부정',
    '피부_긍정','피부_중립','피부_부정',
    '노화_긍정','노화_중립','노화_부정'
]

# 4) 분류 함수 정의
def zsl_label(리뷰, threshold=0.5):
    out = classifier(리뷰, candidate_labels=candidate_labels, multi_label=True)
    return {lab: int(score > threshold) for lab, score in zip(out['labels'], out['scores'])}

# 5) 폴더 내 모든 *_reviews.csv 파일 처리
csv_files = glob.glob("*_reviews.csv")

for file_path in csv_files:
    df = pd.read_csv(file_path)
    tqdm.pandas(desc=f"Zero-Shot 라벨링: {os.path.basename(file_path)}")
    
    # 6) 리뷰 텍스트에 대해 라벨링 수행
    labels_df = df['리뷰내용'].progress_apply(zsl_label).apply(pd.Series)
    
    # 7) 원본 데이터프레임과 합치기
    df_labeled = pd.concat([df, labels_df], axis=1)
    
    # 8) 저장 (파일명_suffix)
    output_file = file_path.replace("_reviews.csv", "_zero_shot_labeled.csv")
    df_labeled.to_csv(output_file, index=False)
    print(f"✅ 저장 완료: {output_file}")

print("🎉 모든 파일 Zero-Shot 라벨링 완료!")


In [22]:
# 5) 폴더 내 모든 *_reviews.csv 파일 처리
csv_files = glob.glob("*_reviews.csv")

for file_path in csv_files:
    df = pd.read_csv(file_path)
    tqdm.pandas(desc=f"Zero-Shot 라벨링: {os.path.basename(file_path)}")
    
    # 6) 리뷰 텍스트에 대해 라벨링 수행 (수정된 함수 사용)
    labels_df = df['리뷰내용'].progress_apply(zsl_label_single_sentiment).apply(pd.Series)
    
    # 7) 원본 데이터프레임과 합치기
    df_labeled = pd.concat([df, labels_df], axis=1)
    
    # 8) 저장 (파일명_suffix)
    output_file = file_path.replace("_reviews.csv", "_zero_shot_labeled.csv")
    df_labeled.to_csv(output_file, index=False)
    print(f"✅ 저장 완료: {output_file}")

print("🎉 모든 파일 Zero-Shot 라벨링 완료!")

Zero-Shot 라벨링: 스킨_토너_reviews.csv:   4%|█▎                                | 1636/44144 [03:30<1:31:01,  7.78it/s]


KeyboardInterrupt: 

In [25]:
import os
import glob
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# 1) 작업 디렉토리로 이동 (파일이 있는 폴더)
os.chdir(r"C:\Users\User\Desktop\skindata")

# 2) Zero-Shot 분류기 로드
classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli",
    device=0  # GPU 사용 시 0으로 변경
)

# 3) 라벨 후보 목록 (15개)
# 각 속성별로 묶어서 관리하는 것이 후처리에 용이합니다.
attribute_labels = {
    '미백': ['미백_긍정', '미백_중립', '미백_부정'],
    '보습': ['보습_긍정', '보습_중립', '보습_부정'],
    '트러블': ['트러블_긍정', '트러블_중립', '트러블_부정'],
    '피부': ['피부_긍정', '피부_중립', '피부_부정'],
    '노화': ['노화_긍정', '노화_중립', '노화_부정']
}

# 모든 후보 라벨을 리스트로 합칩니다.
all_candidate_labels = sum(attribute_labels.values(), [])


# 4) 분류 함수 정의 수정
def zsl_label_single_sentiment(리뷰, threshold=0.5):
    # 이 부분은 각 속성(미백, 보습 등)별로 분류를 수행합니다.
    # multi_label=False로 작동하도록 하여 가장 점수가 높은 라벨만 선택합니다.
    
    results = {}
    for attr, labels in attribute_labels.items():
        # 각 속성별 (예: 미백) 긍정/중립/부정 라벨만 사용하여 분류
        out = classifier(리뷰, candidate_labels=labels, multi_label=False) # <--- multi_label=False 처럼 동작하도록
        
        # 가장 높은 점수를 받은 라벨을 찾습니다.
        best_label = out['labels'][0]
        best_score = out['scores'][0]
        
        # 임계값을 넘으면 해당 라벨에만 1을 부여하고 나머지는 0
        if best_score > threshold:
            results[best_label] = 1
            for lab in labels:
                if lab != best_label:
                    results[lab] = 0
        else: # 임계값을 넘지 않으면 모두 0 (해당 속성에 대한 명확한 언급 없음)
            for lab in labels:
                results[lab] = 0
                
    return results

# 5) 폴더 내 모든 *_reviews.csv 파일 처리
csv_files = glob.glob("*_reviews.csv")

for file_path in csv_files:
    df = pd.read_csv(file_path)
    tqdm.pandas(desc=f"Zero-Shot 라벨링: {os.path.basename(file_path)}")
    
    # 6) 리뷰 텍스트에 대해 라벨링 수행 (수정된 함수 사용)
    labels_df = df['리뷰내용'].progress_apply(zsl_label_single_sentiment).apply(pd.Series)
    
    # 7) 원본 데이터프레임과 합치기
    df_labeled = pd.concat([df, labels_df], axis=1)
    
    # 8) 저장 (파일명_suffix)
    output_file = file_path.replace("_reviews.csv", "_zero_shot_labeled.csv")
    df_labeled.to_csv(output_file, index=False)
    print(f"✅ 저장 완료: {output_file}")

print("🎉 모든 파일 Zero-Shot 라벨링 완료!")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda:0
Zero-Shot 라벨링: 스킨_토너_reviews.csv: 100%|█████████████████████████████████| 44144/44144 [1:28:05<00:00,  8.35it/s]


✅ 저장 완료: 스킨_토너_zero_shot_labeled.csv


Zero-Shot 라벨링: 아이크림_reviews.csv: 100%|██████████████████████████████████| 32106/32106 [1:03:26<00:00,  8.43it/s]


✅ 저장 완료: 아이크림_zero_shot_labeled.csv


Zero-Shot 라벨링: 에센스_세럼_앰플_reviews.csv: 100%|██████████████████████████| 46326/46326 [1:32:54<00:00,  8.31it/s]


...

✅ 저장 완료: 워터_밀크_zero_shot_labeled.csv


Zero-Shot 라벨링: 크림_reviews.csv: 100%|██████████████████████████████████████| 42560/42560 [1:25:14<00:00,  8.32it/s]


✅ 저장 완료: 크림_zero_shot_labeled.csv


Zero-Shot 라벨링: 클렌징폼_젤_reviews.csv: 100%|███████████████████████████████| 46568/46568 [1:33:25<00:00,  8.31it/s]


✅ 저장 완료: 클렌징폼_젤_zero_shot_labeled.csv
🎉 모든 파일 Zero-Shot 라벨링 완료!


In [27]:
import glob
import pandas as pd
from tqdm.auto import tqdm

# 1) 합칠 파일 목록 가져오기
labeled_files = glob.glob("*_zero_shot_labeled.csv")

# 2) 각 파일을 읽어서 리스트에 담기
dfs = []
for fp in tqdm(labeled_files, desc="Reading labeled files"):
    df = pd.read_csv(fp)
    dfs.append(df)

# 3) 세로로 합치기
combined = pd.concat(dfs, axis=0, ignore_index=True)

# 4) 결과 저장
combined.to_csv("labeled.csv", index=False)
print(f"✅ {len(dfs)}개 파일 합쳐서 labeled.csv 저장 완료 (총 {len(combined)}개 행)") 


Reading labeled files: 100%|█████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  2.75it/s]


✅ 7개 파일 합쳐서 labeled.csv 저장 완료 (총 297370개 행)
